In [ ]:
import basicModel as bm
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
#股票信息数据
infodf = pd.read_csv('./database/infoall.csv').drop(['Unnamed: 0.1'],axis=1).set_index('Unnamed: 0')
infodf.index.name = 'code'
newsdf = pd.read_csv('./database/preopen_news.csv').drop(['备注'],axis=1)

In [ ]:
#模型初始化
fb = bm.basicmodel()
fb.initialize()

In [ ]:
#获取交易日
close = pd.read_feather('./database/BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
close = close['2022-07-01':]
tradeday = close.index
#交易日股票的交易状态
tradestatus = close.mask(close>0,1)

In [ ]:
for tradedate in tradeday[:1]:
    #选择交易日对应的全部热搜按照最高排名排序，结果的前十条
    daynewsdf = newsdf[newsdf['交易日']==tradeday[1].date().isoformat()].sort_values('最高排名').head(20)
    #选取当日正常开盘的股票
    dayinfo = infodf['introduction'][tradestatus.loc[tradedate].dropna().index]
    day_high_simlist = []
    for news in daynewsdf.sort_values('最高排名').head(2).itertuples():
        simlist = []
        for stockinfo in tqdm(dayinfo.dropna().items()):
            #计算相似度，文本上限500（模型设定上限512）
            sim = fb.calc_similarity(news[2],stockinfo[1][:500])
            simlist.append((stockinfo[0],stockinfo[1],sim))
        simlist.sort(key=lambda x:x[2])
        #相似度最高的五只股票
        day_high_simlist.append(simlist[-5:])